In [1]:
import sqlite3
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier

In [22]:
conn = sqlite3.connect('C:\\Users\\shash\\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2\\policeResourceAllocation\\Shared DC2 Group 19\\database_final.db')

query_burg = """SELECT * FROM table_name
WHERE "Crime Type" = "Burglary"
"""
query_sun = """SELECT * FROM sunlight"""
query_house = """SELECT * FROM housing_by_lsoa"""

df_burglary = pd.read_sql(query_burg, conn)
df_sunlight = pd.read_sql(query_sun, conn)
df_housing = pd.read_sql(query_house, conn)

conn.close()

In [27]:
"""file_path ='Shared DC2 Group 19\\Cleaned Data\\conc_clean.csv'
df_burglary = pd.read_csv(file_path)"""

In [33]:
df_burglary = df_burglary.drop(columns=['Longitude', 'Latitude', 'Crime type', 'Last outcome category', 'Context']).dropna()



In [39]:
df_bar=df_burglary[df_burglary['LSOA name'].str.contains('Barnet')]
df_bar['Month'] = pd.to_datetime(df_bar['Month'])
df_bar['Year']= df_bar['Month'].dt.year
df_bar_2019 =df_bar[ df_bar['Year'] == 2019]
df_bar_2019

C:\Users\shash\AppData\Local\Temp\ipykernel_2464\3195209047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bar['Month'] = pd.to_datetime(df_bar['Month'])
C:\Users\shash\AppData\Local\Temp\ipykernel_2464\3195209047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bar['Year']= df_bar['Month'].dt.year


,Month,Reported by,Falls within,Location,LSOA code,LSOA name,Crime type,Year
8359537,2019-01-01,Metropolitan Police Service,Metropolitan Police Service,On or near Montague Close,E01000250,Barnet 001C,Burglary,2019
8359538,2019-01-01,Metropolitan Police Service,Metropolitan Police Service,On or near High Street,E01000250,Barnet 001C,Burglary,2019
8359570,2019-01-01,Metropolitan Police Service,Metropolitan Police Service,On or near Taylor'S Lane,E01000254,Barnet 001D,Burglary,2019
8359571,2019-01-01,Metropolitan Police Service,Metropolitan Police Service,On or near Hadley Highstone,E01000254,Barnet 001D,Burglary,2019
8359575,2019-01-01,Metropolitan Police Service,Metropolitan Police Service,On or near Further/Higher Educational Building,E01000292,Barnet 001E,Burglary,2019
...,...,...,...,...,...,...,...,...
9371923,2019-12-01,Metropolitan Police Service,Metropolitan Police Service,On or near Purbeck Drive,E01000224,Barnet 040D,Burglary,2019
9371940,2019-12-01,Metropolitan Police Service,Metropolitan Police Service,On or near Hocroft Walk,E01000139,Barnet 041B,Burglary,2019
9371957,2019-12-01,Metropolitan Police Service,Metropolitan Police Service,On or near Prospect Road,E01000140,Barnet 041C,Burglary,2019
9371958,2019-12-01,Metropolitan Police Service,Metropolitan Police Service,On or near Elm Terrace,E01000140,Barnet 041C,Burglary,2019


In [53]:
df_sunlight['Year-Month'] = pd.to_datetime(df_sunlight['Year-Month'])
df_sunlight_2019 = df_sunlight[df_sunlight['Year-Month'].dt.year == 2019]
#df_sunlight_2019['Month']= df_sunlight['Year-Month'].dt.month
df_sunlight_2019['Month']= df_sunlight['Year-Month']
df_sunlight_2019

C:\Users\shash\AppData\Local\Temp\ipykernel_2464\1259476284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sunlight_2019['Month']= df_sunlight['Year-Month']


,index,Sunlight,Year-Month,Month
216,216,1.740893,2019-01-01,2019-01-01
217,217,4.363079,2019-02-01,2019-02-01
218,218,4.164106,2019-03-01,2019-03-01
219,219,5.779824,2019-04-01,2019-04-01
220,220,6.574293,2019-05-01,2019-05-01
221,221,5.710219,2019-06-01,2019-06-01
222,222,6.829995,2019-07-01,2019-07-01
223,223,6.713303,2019-08-01,2019-08-01
224,224,5.423063,2019-09-01,2019-09-01
225,225,2.654354,2019-10-01,2019-10-01


In [64]:
# Load the cleaned data into a dataframe
df = df_bar_2019

# Convert the 'Month' column to a pandas datetime object
df['Month'] = pd.to_datetime(df['Month'])

# Filter the dataframe to only include rows with 'Burglary' in the 'Crime type' column
burglary_df = df[df['Crime type'] == 'Burglary']

# Create a new dataframe with every month of all years
all_months = pd.DataFrame(pd.date_range(start='2018-12-01', end='2020-01-01', freq='MS'), columns=['Month'])

# Group the 'Burglary' dataframe by month and count the number of rows in each month
monthly_counts = burglary_df.groupby(pd.Grouper(key='Month', freq='MS'))['Crime type'].count()

# Combine the 'all_months' dataframe and the 'monthly_counts' series
combined_df_copy = pd.merge(all_months, pd.DataFrame(monthly_counts), how='left', left_on='Month', right_on='Month')

# Fill any missing values with 0
combined_df_copy.fillna(0, inplace=True)

# Rename the 'Crime type' column to 'Number of Burglaries'
combined_df_copy.rename(columns={'Crime type': 'Number of Burglaries'}, inplace=True)

# Set the 'Month' column as the index
combined_df_copy.set_index('Month', inplace=True)
combined_df_copy = combined_df_copy.reset_index()

# Preview the resulting dataframe
combined_df_copy.head()

C:\Users\shash\AppData\Local\Temp\ipykernel_2464\1650166311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = pd.to_datetime(df['Month'])


,Month,Number of Burglaries
0,2018-12-01,0.0
1,2019-01-01,317.0
2,2019-02-01,336.0
3,2019-03-01,328.0
4,2019-04-01,286.0


In [130]:
temp_merged = pd.merge(df_bar_2019, df_housing, how='inner', on=['LSOA code', 'LSOA name'])
temp_merged
df_merged = pd.merge(temp_merged, df_sunlight_2019, how='inner', on=['Month'])
temp_merged
temp_merged_clean = temp_merged.drop(columns=['Reported by','Falls within','Year','Crime type','Missing', 'index'])
temp_merged_clean
df_final = pd.merge(temp_merged_clean, combined_df_copy, how='inner', on=['Month'])
df_final

,Month,Location,LSOA code,LSOA name,Owned or shared ownership,Private Rented or lives rent free,Social Rented,Proportion social,Proportion rented,Detached whole house or bungalow,...,Terraced (including end-terrace) whole house or bungalow,"Flat, maisonette or apartment",A caravan or other mobile or temporary structure,Proportion of indep,1 bedroom,2 bedrooms,3 bedrooms,4 or more bedrooms,Proportion small house,Number of Burglaries
0,2019-01-01,On or near Montague Close,E01000250,Barnet 001C,575,415,155,0.135371,0.497817,90,...,105,700,0,0.414226,310,350,270,215,0.576419,317.0
1,2019-01-01,On or near High Street,E01000250,Barnet 001C,575,415,155,0.135371,0.497817,90,...,105,700,0,0.414226,310,350,270,215,0.576419,317.0
2,2019-01-01,On or near Taylor'S Lane,E01000254,Barnet 001D,500,115,20,0.031496,0.212598,170,...,265,85,0,0.871212,35,165,200,235,0.314961,317.0
3,2019-01-01,On or near Hadley Highstone,E01000254,Barnet 001D,500,115,20,0.031496,0.212598,170,...,265,85,0,0.871212,35,165,200,235,0.314961,317.0
4,2019-01-01,On or near Further/Higher Educational Building,E01000292,Barnet 001E,455,135,60,0.092308,0.300000,105,...,65,270,0,0.605839,95,170,230,160,0.404580,317.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3413,2019-12-01,On or near Walden Way,E01000257,Barnet 017B,430,215,265,0.291209,0.527473,35,...,70,830,0,0.186275,205,490,160,55,0.763736,346.0
3414,2019-12-01,On or near Field Mead,E01000152,Barnet 026C,445,145,340,0.365591,0.521505,0,...,165,685,0,0.267380,125,475,260,70,0.645161,346.0
3415,2019-12-01,On or near Further/Higher Educational Building,E01000152,Barnet 026C,445,145,340,0.365591,0.521505,0,...,165,685,0,0.267380,125,475,260,70,0.645161,346.0
3416,2019-12-01,On or near Grahame Park Way,E01000152,Barnet 026C,445,145,340,0.365591,0.521505,0,...,165,685,0,0.267380,125,475,260,70,0.645161,346.0


In [138]:
temp_merged_clean_grp = df_final.groupby("LSOA code").sum()
temp_merged_clean_grp['Risk'] = 0
temp_merged_clean_grp.loc[temp_merged_clean_grp['Number of Burglaries'] > 1000, 'Risk'] = 0
temp_merged_clean_grp.loc[temp_merged_clean_grp['Number of Burglaries'] > 3500, 'Risk'] = 1  
temp_merged_clean_grp.loc[temp_merged_clean_grp['Number of Burglaries'] > 6000, 'Risk'] = 2
temp_merged_clean_grp.loc[temp_merged_clean_grp['Number of Burglaries'] > 8000, 'Risk'] = 3
temp_merged_clean_grp = temp_merged_clean_grp.drop(columns=['Number of Burglaries'])
temp_merged_clean_grp

C:\Users\shash\AppData\Local\Temp\ipykernel_2464\1216932731.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp_merged_clean_grp = df_final.groupby("LSOA code").sum()


,Owned or shared ownership,Private Rented or lives rent free,Social Rented,Proportion social,Proportion rented,Detached whole house or bungalow,Semi-detached whole house or bungalow,Terraced (including end-terrace) whole house or bungalow,"Flat, maisonette or apartment",A caravan or other mobile or temporary structure,Proportion of indep,1 bedroom,2 bedrooms,3 bedrooms,4 or more bedrooms,Proportion small house,Risk
LSOA code,,,,,,,,,,,,,,,,,
E01000115,5220,540,0,0.000000,1.125000,1620,4020,0,180,0,11.628866,0,120,2460,3060,0.255319,1
E01000116,4275,855,405,0.658537,2.048780,135,2295,2565,675,0,7.928571,495,270,2880,1890,1.243902,0
E01000117,10640,2375,0,0.000000,3.467153,475,7600,2565,3040,0,14.777778,475,3800,5700,3135,6.195652,2
E01000118,10890,2340,360,0.476821,3.576159,180,6390,4590,2700,0,14.493506,810,2160,6660,3870,3.960000,1
E01000119,3840,840,320,0.512000,1.856000,520,3040,1280,240,0,7.622047,160,440,2560,1880,0.952381,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E01000320,7875,3525,750,0.925926,5.277778,825,2700,2625,6525,0,7.278107,2775,3375,2625,3375,7.592593,1
E01000321,5325,6225,600,0.740741,8.425926,225,1500,3075,7875,0,5.680473,3300,4425,2325,2025,9.596273,1
E01000322,7875,7770,1260,1.565217,11.217391,735,2415,525,13650,0,4.454545,3150,9660,1890,2205,15.913043,1


In [139]:
from sklearn.model_selection import train_test_split

X = temp_merged_clean_grp.drop(columns=['Risk'])
y = temp_merged_clean_grp['Risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [140]:
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier

estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBClassifier(random_state=8)) # can customize objective function with the objective parameter
]
pipe = Pipeline(steps=estimators)
pipe

Pipeline(steps=[('encoder', TargetEncoder()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=8, ...))])

In [141]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=3, n_iter=10, scoring='roc_auc', random_state=8) 

In [142]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [143]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print ("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.61%


: 